In [106]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [107]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [108]:
from sklearn.cross_validation import train_test_split

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [110]:
import lightgbm as lgb

In [163]:
estimator = lgb.LGBMClassifier(num_leaves=2)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.05, 0.5, 1],
    'n_estimators': [20, 40, 60, 80, 100, 120]
}

gbm = GridSearchCV(estimator, param_grid, cv=5, scoring='roc_auc')

gbm.fit(X, y)

print('Best parameters found by grid search are:', gbm.best_params_)


('Best parameters found by grid search are:', {'n_estimators': 80, 'learning_rate': 0.5})


In [164]:
sum(gbm.best_estimator_.predict(X_test) == y_test) / (len(y_test)*1.0)

0.82397003745318353

In [165]:
sum(
    (gbm.best_estimator_.predict_proba(X_test)[:,1]> 0.67 ) == y_test) / (len(y_test)*1.0)

0.86142322097378277

In [166]:
from xgboost.sklearn import XGBClassifier
estimator = XGBClassifier()

In [167]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

In [170]:
clf = GridSearchCV(estimator, parameters, n_jobs=5, 
                   cv=5, 
                   scoring='roc_auc',
                   verbose=2, refit=True)

clf.fit(X, y)
print('Best parameters found by grid search are:', gbm.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    1.6s remaining:    2.4s


('Best parameters found by grid search are:', {'n_estimators': 80, 'learning_rate': 0.5})


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    2.7s finished


In [171]:
sum(clf.best_estimator_.predict(X_test) == y_test) / (len(y_test)*1.0)

0.8202247191011236

In [180]:
sum(
    (clf.best_estimator_.predict_proba(X_test)[:,1]> 0.51 ) == y_test) / (len(y_test)*1.0)

0.85018726591760296